In [1]:
import pandas as pd
import cPickle as pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
import scipy
# from scikits import bootstrap as boot
sns.set(font_scale=1.0)
from pandas.plotting import table
import custom_image_toolkit as c
from scipy.stats import ttest_ind

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
# pACT1-mKate2 experiment on 180910
# expt_id = '/180910_yFB71'
path = './expt_ids'
cell_type_index={'M':'Mothers','D':'Daughters'}
# # yFB79 Raffinose experiment on 181207
expt_id1 = '/181207_yFB79_60X_Raff_125uMGal'
df1 = pd.read_pickle(path+expt_id1+".pkl")
df1['expt'] = expt_id1
df1['label'] = 'pWHI5-WHI5 12/07'
df1['genotype'] = 'pWHI5-WHI5'
# yFB79 Raffinose experiment on 190417
expt_id2 = '/190417_yFB79_60X_Raff_125uMGal'
df2= pd.read_pickle(path+expt_id2+".pkl")
df2['expt'] = '/190417_yFB79_60X_Raff_125uMGal'
df2['label'] = 'pWHI5-WHI5 4/17'
df2['genotype'] = 'pWHI5-WHI5'
# yFB78 raffinose on 190606
expt_id0 = '/190606_yFB78_60X_Raff_125uMGal'
df0 = pd.read_pickle(path+expt_id0+".pkl")
df0['expt'] = expt_id0
df0['label'] = 'pGAL1-WHI5 06/06'
df0['genotype'] = 'pGAL1-WHI5'
# yFB78 expt 190607, 12 min timestep
expt_id3 = '/190607_yFB78_60X_Raff_125uMGal'
df3= pd.read_pickle(path+expt_id3+".pkl")
df3['expt'] = '/190607_yFB78_60X_Raff_125uMGal'
df3['label'] = 'pGAL1-WHI5 6/07'
df3['genotype'] = 'pGAL1-WHI5'
# yFB78 expt 190725
expt_id4 = '/190725_yFB78_60X_2Raff_125uMGal'
df4= pd.read_pickle(path+expt_id4+".pkl")
df4['expt'] = expt_id4
df4['label'] = 'pGAL1-WHI5 7/25'
df4['genotype'] = 'pGAL1-WHI5'
# yFB79 Raffinose experiment on 190612
expt_id6 = '/190612_yFB79_timelapse'
df6= pd.read_pickle(path+expt_id6+".pkl")
df6['expt'] = expt_id6
df6['label'] = 'pWHI5-WHI5 6/12'
df6['genotype'] = 'pWHI5-WHI5'

df0=c.processing(df0)
df1=c.processing(df1)
df=df0.append(df1);
df2=c.processing(df2)
df=df.append(df2);
df3=c.processing(df3)
df=df.append(df3);
df4=c.processing(df4)
df=df.append(df4);
df6=c.processing(df6)
df=df.append(df6);
df = c.normalize(df);
df['Cell Type'] = [cell_type_index[df.iloc[i0].type] for i0 in range(len(df))]
df['Condition'] = [cell_type_index[df.iloc[i0].type]+', '+df.iloc[i0].genotype for i0 in range(len(df))]

In [5]:
# df.to_pickle(path+'/compiled_timelapse_data.pkl')

In [6]:
df.groupby(['Cell Type', 'genotype']).describe()

$t_{G1}$                                            \
                        count        mean        std   min   25%    50%   
Cell Type genotype                                                        
Daughters pGAL1-WHI5    347.0  112.155620  66.246597  10.0  60.0  100.0   
          pWHI5-WHI5    853.0  123.484174  52.113692  24.0  80.0  120.0   
Mothers   pGAL1-WHI5    800.0   36.735000  37.174820  10.0  12.0   24.0   
          pWHI5-WHI5   1581.0   40.493359  38.639777  10.0  20.0   30.0   

                                   $t_{b}$              ... $c1_{s,seg,norm}$  \
                        75%    max   count        mean  ...               75%   
Cell Type genotype                                      ...                     
Daughters pGAL1-WHI5  156.0  396.0   347.0  249.631124  ...          1.015121   
          pWHI5-WHI5  160.0  330.0   853.0  229.826495  ...          0.869775   
Mothers   pGAL1-WHI5   40.0  228.0   800.0  276.982500  ...          0.905209   
          pWHI5-WHI5   40.0  348.0  1581.0  306.894371  ...          0.847621   

                               $c1_{div,seg,norm}$                      \
                           max               count      mean       std   
Cell Type genotype                                                       
Daughters pGAL1-WHI5  2.494785               347.0  1.246853  0.468142   
          pWHI5-WHI5  1.377727               853.0  1.555784  0.311530   
Mothers   pGAL1-WHI5  4.552213               800.0       inf       NaN   
          pWHI5-WHI5  1.362617              1581.0       inf       NaN   

                                                                        
                           min       25%       50%       75%       max  
Cell Type genotype                                                      
Daughters pGAL1-WHI5  0.415158  0.900422  1.157249  1.517383  3.086848  
          pWHI5-WHI5  0.795012  1.345036  1.518488  1.723013  3.372347  
Mothers   pGAL1-WHI5  0.323959  0.966701  1.254893  1.703216       inf  
          pWHI5-WHI5  0.459202  1.392868  1.561677  1.773276       inf  

[4 rows x 488 columns]

In [13]:
var = "$c1_{b,seg,norm}$"
temp_df= df.groupby(['Condition', 'expt'])[var].mean()
labels = temp_df.index.unique().levels[0]
# print temp_df.head(20)
print 'P values for average of '+var
# temp_df.loc['Daughters, pWHI5-WHI5']
for i in range(len(labels)):
    for j in range(len(labels)):
        if i<j:
            print labels[i], labels[j]
            vals= scipy.stats.ttest_ind(temp_df.loc[labels[i]],temp_df.loc[labels[j]],equal_var=False)
            
            if vals[1]<0.05:
                print 'Statistically significant difference'
                print np.around(vals,3)
            else:
                print 'No difference'

P values for average of $c1_{b,seg,norm}$
Daughters, pGAL1-WHI5 Daughters, pWHI5-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pGAL1-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference
Daughters, pWHI5-WHI5 Mothers, pGAL1-WHI5
No difference
Daughters, pWHI5-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[7.757e+00 2.000e-03]
Mothers, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference


In [14]:
var = "$c2_{b,seg,norm}$"
temp_df= df.groupby(['Condition', 'expt'])[var].mean()
labels = temp_df.index.unique().levels[0]
# print temp_df.head(20)
print 'P values for average of '+var
# temp_df.loc['Daughters, pWHI5-WHI5']
for i in range(len(labels)):
    for j in range(len(labels)):
        if i<j:
            print labels[i], labels[j]
            vals= scipy.stats.ttest_ind(temp_df.loc[labels[i]],temp_df.loc[labels[j]],equal_var=False)
            
            if vals[1]<0.05:
                print 'Statistically significant difference'
                print np.around(vals,3)
            else:
                print 'No difference'

P values for average of $c2_{b,seg,norm}$
Daughters, pGAL1-WHI5 Daughters, pWHI5-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pGAL1-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference
Daughters, pWHI5-WHI5 Mothers, pGAL1-WHI5
Statistically significant difference
[6.043e+00 4.000e-03]
Daughters, pWHI5-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[4.359 0.016]
Mothers, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference


In [8]:
var = "$c2_{b,seg,norm}$"
temp_df= df.groupby(['Condition', 'expt'])[var].mean()
labels = temp_df.index.unique().levels[0]
# print temp_df.head(20)
print 'P values for average of '+var
# temp_df.loc['Daughters, pWHI5-WHI5']
for i in range(len(labels)):
    for j in range(len(labels)):
        if i<j:
            print labels[i], labels[j]
            vals= scipy.stats.ttest_ind(temp_df.loc[labels[i]],temp_df.loc[labels[j]],equal_var=False)
            
            if vals[1]<0.05:
                print 'Statistically significant difference'
                print np.around(vals,3)
            else:
                print 'No difference'

P values for average of $c2_{b,seg,norm}$
Daughters, pGAL1-WHI5 Daughters, pWHI5-WHI5
Statistically significant difference
[-3.901  0.038]
Daughters, pGAL1-WHI5 Mothers, pGAL1-WHI5
Statistically significant difference
[12.716  0.   ]
Daughters, pGAL1-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[14.032  0.   ]
Daughters, pWHI5-WHI5 Mothers, pGAL1-WHI5
Statistically significant difference
[1.0789e+01 3.0000e-03]
Daughters, pWHI5-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[1.2456e+01 1.0000e-03]
Mothers, pGAL1-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[3.847 0.023]


In [16]:
var = "$t_{G1}$"
temp_df= df.groupby(['Condition', 'expt'])[var].mean()
labels = temp_df.index.unique().levels[0]
# print temp_df.head(20)
print 'P values for average of '+var
# temp_df.loc['Daughters, pWHI5-WHI5']
for i in range(len(labels)):
    for j in range(len(labels)):
        if i<j:
            print labels[i], labels[j]
            vals= scipy.stats.ttest_ind(temp_df.loc[labels[i]],temp_df.loc[labels[j]],equal_var=False)
            
            if vals[1]<0.1:
                print 'Statistically significant difference'
                print np.around(vals,3)
            else:
                print 'No difference'

P values for average of $t_{G1}$
Daughters, pGAL1-WHI5 Daughters, pWHI5-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pGAL1-WHI5
Statistically significant difference
[3.741 0.062]
Daughters, pGAL1-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[3.543 0.068]
Daughters, pWHI5-WHI5 Mothers, pGAL1-WHI5
Statistically significant difference
[9.824e+00 7.000e-03]
Daughters, pWHI5-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[9.344e+00 7.000e-03]
Mothers, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference


In [15]:
var = "$t_{div}$"
temp_df= df.groupby(['Condition', 'expt'])[var].mean()
labels = temp_df.index.unique().levels[0]
# print temp_df.head(20)
print 'P values for average of '+var
# temp_df.loc['Daughters, pWHI5-WHI5']
for i in range(len(labels)):
    for j in range(len(labels)):
        if i<j:
            print 'Comparing: ',labels[i], labels[j]
            vals= scipy.stats.ttest_ind(temp_df.loc[labels[i]],temp_df.loc[labels[j]],equal_var=False)
            if vals[1]<0.1:
                print 'Statistically significant difference'
                print np.around(vals,3)
            else:
                print 'No difference'

P values for average of $t_{div}$
Comparing:  Daughters, pGAL1-WHI5 Daughters, pWHI5-WHI5
No difference
Comparing:  Daughters, pGAL1-WHI5 Mothers, pGAL1-WHI5
Statistically significant difference
[4.039 0.029]
Comparing:  Daughters, pGAL1-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[5.027 0.027]
Comparing:  Daughters, pWHI5-WHI5 Mothers, pGAL1-WHI5
Statistically significant difference
[6.119e+00 5.000e-03]
Comparing:  Daughters, pWHI5-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[7.924e+00 6.000e-03]
Comparing:  Mothers, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference


In [12]:
var = "$t_{div}$"
temp_df= df.groupby(['Condition', 'expt'])[var].mean()
labels = temp_df.index.unique().levels[0]
# print temp_df.head(20)
print 'P values for average of '+var
# temp_df.loc['Daughters, pWHI5-WHI5']
for i in range(len(labels)):
    for j in range(len(labels)):
        if i<j:
            print 'Comparing: ',labels[i], labels[j]
            vals= scipy.stats.ttest_ind(temp_df.loc[labels[i]],temp_df.loc[labels[j]],equal_var=False)
            if vals[1]<0.1:
                print 'Statistically significant difference'
                print np.around(vals,3)
            else:
                print 'No difference'

P values for average of $t_{div}$
Comparing:  Daughters, pGAL1-WHI5 Daughters, pWHI5-WHI5
No difference
Comparing:  Daughters, pGAL1-WHI5 Mothers, pGAL1-WHI5
Statistically significant difference
[4.039 0.029]
Comparing:  Daughters, pGAL1-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[5.027 0.027]
Comparing:  Daughters, pWHI5-WHI5 Mothers, pGAL1-WHI5
Statistically significant difference
[6.119e+00 5.000e-03]
Comparing:  Daughters, pWHI5-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[7.924e+00 6.000e-03]
Comparing:  Mothers, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference


In [13]:
var = "$t_{budded}$"
temp_df= df.groupby(['Condition', 'expt'])[var].mean()
labels = temp_df.index.unique().levels[0]
# print temp_df.head(20)
# temp_df.loc['Daughters, pWHI5-WHI5']
print 'P values for average of '+var
for i in range(len(labels)):
    for j in range(len(labels)):
        if i<j:
            print labels[i], labels[j]
            vals= scipy.stats.ttest_ind(temp_df.loc[labels[i]],temp_df.loc[labels[j]],equal_var=False)
            if vals[1]<0.1:
                print 'Statistically significant difference'
                print np.around(vals,3)
            else:
                print 'No difference'

P values for average of $t_{budded}$
Daughters, pGAL1-WHI5 Daughters, pWHI5-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pGAL1-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference
Daughters, pWHI5-WHI5 Mothers, pGAL1-WHI5
No difference
Daughters, pWHI5-WHI5 Mothers, pWHI5-WHI5
No difference
Mothers, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference


In [7]:
var = "$V_{b,ell}$"
temp_df= df.groupby(['Condition', 'expt'])[var].std()/df.groupby(['Condition', 'expt'])[var].mean()
labels = temp_df.index.unique().levels[0]
# print temp_df.head(20)
# temp_df.loc['Daughters, pWHI5-WHI5']
print 'P values for CV of '+var
for i in range(len(labels)):
    for j in range(len(labels)):
        if i<j:
            print labels[i], labels[j]
            vals= scipy.stats.ttest_ind(temp_df.loc[labels[i]],temp_df.loc[labels[j]],equal_var=False)
            if vals[1]<0.1:
                print 'Statistically significant difference'
                print np.around(vals,3)
            else:
                print 'No difference'

P values for CV of $V_{b,ell}$
Daughters, pGAL1-WHI5 Daughters, pWHI5-WHI5
Statistically significant difference
[4.774 0.032]
Daughters, pGAL1-WHI5 Mothers, pGAL1-WHI5
Statistically significant difference
[-2.978  0.041]
Daughters, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference
Daughters, pWHI5-WHI5 Mothers, pGAL1-WHI5
Statistically significant difference
[-8.339  0.01 ]
Daughters, pWHI5-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[-4.151  0.05 ]
Mothers, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference


In [8]:
var = "$V_{b,ell}$"
temp_df= df.groupby(['Condition', 'expt'])[var].mean()
labels = temp_df.index.unique().levels[0]
# print temp_df.head(20)
# temp_df.loc['Daughters, pWHI5-WHI5']
print 'P values for av of '+var
for i in range(len(labels)):
    for j in range(len(labels)):
        if i<j:
            print labels[i], labels[j]
            vals= scipy.stats.ttest_ind(temp_df.loc[labels[i]],temp_df.loc[labels[j]],equal_var=False)
            if vals[1]<0.1:
                print 'Statistically significant difference'
                print np.around(vals,3)
            else:
                print 'No difference'

P values for av of $V_{b,ell}$
Daughters, pGAL1-WHI5 Daughters, pWHI5-WHI5
Statistically significant difference
[4.104 0.028]
Daughters, pGAL1-WHI5 Mothers, pGAL1-WHI5
Statistically significant difference
[-3.631  0.067]
Daughters, pGAL1-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[-5.661  0.028]
Daughters, pWHI5-WHI5 Mothers, pGAL1-WHI5
Statistically significant difference
[-4.017  0.056]
Daughters, pWHI5-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[-6.159  0.025]
Mothers, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference


In [9]:
var = "$V_{s,ell}$"
temp_df= df.groupby(['Condition', 'expt'])[var].std()/df.groupby(['Condition', 'expt'])[var].mean()
labels = temp_df.index.unique().levels[0]
# print temp_df.head(20)
# temp_df.loc['Daughters, pWHI5-WHI5']
print 'P values for CV of '+var
for i in range(len(labels)):
    for j in range(len(labels)):
        if i<j:
            print labels[i], labels[j]
            vals= scipy.stats.ttest_ind(temp_df.loc[labels[i]],temp_df.loc[labels[j]],equal_var=False)
            if vals[1]<0.1:
                print 'Statistically significant difference'
                print np.around(vals,3)
            else:
                print 'No difference'

P values for CV of $V_{s,ell}$
Daughters, pGAL1-WHI5 Daughters, pWHI5-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pGAL1-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference
Daughters, pWHI5-WHI5 Mothers, pGAL1-WHI5
No difference
Daughters, pWHI5-WHI5 Mothers, pWHI5-WHI5
No difference
Mothers, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference


In [10]:
var = "$V_{s,ell}$"
temp_df= df.groupby(['Condition', 'expt'])[var].mean()
labels = temp_df.index.unique().levels[0]
# print temp_df.head(20)
# temp_df.loc['Daughters, pWHI5-WHI5']
print 'P values for av of '+var
for i in range(len(labels)):
    for j in range(len(labels)):
        if i<j:
            print labels[i], labels[j]
            vals= scipy.stats.ttest_ind(temp_df.loc[labels[i]],temp_df.loc[labels[j]],equal_var=False)
            if vals[1]<0.1:
                print 'Statistically significant difference'
                print np.around(vals,3)
            else:
                print 'No difference'

P values for av of $V_{s,ell}$
Daughters, pGAL1-WHI5 Daughters, pWHI5-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pGAL1-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[-2.318  0.085]
Daughters, pWHI5-WHI5 Mothers, pGAL1-WHI5
No difference
Daughters, pWHI5-WHI5 Mothers, pWHI5-WHI5
Statistically significant difference
[-3.092  0.07 ]
Mothers, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference


In [11]:
var = "$V_{div,ell}$"
temp_df= df.groupby(['Condition', 'expt'])[var].std()/df.groupby(['Condition', 'expt'])[var].mean()
labels = temp_df.index.unique().levels[0]
# print temp_df.head(20)
# temp_df.loc['Daughters, pWHI5-WHI5']
print 'P values for CV of '+var
for i in range(len(labels)):
    for j in range(len(labels)):
        if i<j:
            print labels[i], labels[j]
            vals= scipy.stats.ttest_ind(temp_df.loc[labels[i]],temp_df.loc[labels[j]],equal_var=False)
            if vals[1]<0.1:
                print 'Statistically significant difference'
                print np.around(vals,3)
            else:
                print 'No difference'

P values for CV of $V_{div,ell}$
Daughters, pGAL1-WHI5 Daughters, pWHI5-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pGAL1-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference
Daughters, pWHI5-WHI5 Mothers, pGAL1-WHI5
No difference
Daughters, pWHI5-WHI5 Mothers, pWHI5-WHI5
No difference
Mothers, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference


In [12]:
var = "$V_{div,ell}$"
temp_df= df.groupby(['Condition', 'expt'])[var].mean()
labels = temp_df.index.unique().levels[0]
# print temp_df.head(20)
# temp_df.loc['Daughters, pWHI5-WHI5']
print 'P values for av of '+var
for i in range(len(labels)):
    for j in range(len(labels)):
        if i<j:
            print labels[i], labels[j]
            vals= scipy.stats.ttest_ind(temp_df.loc[labels[i]],temp_df.loc[labels[j]],equal_var=False)
            if vals[1]<0.1:
                print 'Statistically significant difference'
                print np.around(vals,3)
            else:
                print 'No difference'

P values for av of $V_{div,ell}$
Daughters, pGAL1-WHI5 Daughters, pWHI5-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pGAL1-WHI5
No difference
Daughters, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference
Daughters, pWHI5-WHI5 Mothers, pGAL1-WHI5
No difference
Daughters, pWHI5-WHI5 Mothers, pWHI5-WHI5
No difference
Mothers, pGAL1-WHI5 Mothers, pWHI5-WHI5
No difference


In [5]:
df['expt'].unique()

array(['/190606_yFB78_60X_Raff_125uMGal',
       '/181207_yFB79_60X_Raff_125uMGal',
       '/190417_yFB79_60X_Raff_125uMGal',
       '/190607_yFB78_60X_Raff_125uMGal',
       '/190725_yFB78_60X_2Raff_125uMGal', '/190612_yFB79_timelapse'],
      dtype=object)

In [5]:
def fn(x,slope1,slope2,av1,av2,av3,av4,av5,av6,av7):
    # av1 is the basic x intercept
    # av2 is the diff in the x intercept for genotype 1
    # av3-av8 are the diffs in the x intercepts for the different rns
    # slope1 is the slope for the simple linear regression
    # slope2 is the slope for the genotype*x linear regression
#     temp_cols=x.columns # should be in order yvar, xvar, genotype, runs1-6 in numbers 3-8
    return slope1*x[0]+slope2*x[0]*x[1]+av1+av2*x[1]+av3*x[2]+av4*x[3]+av5*x[4]+av6*x[5]+av7*x[6]

# def fn(x,slope1,slope2,av1,av2,av3,av4,av5):
#     # av1 is the basic x intercept
#     # av2 is the diff in the x intercept for genotype 1
#     # av3-av8 are the diffs in the x intercepts for the different rns
#     # slope1 is the slope for the simple linear regression
#     # slope2 is the slope for the genotype*x linear regression
# #     temp_cols=x.columns # should be in order yvar, xvar, genotype, runs1-6 in numbers 3-8
#     # negative values here are to ensure that the intercepts add up to zero over multiple runs, so that 
#     # we get a robust sense of the overall average
#     return slope1*x[0]+slope2*x[0]*x[1]+av1+av2*x[1]+av3*x[2]-av3*x[3]+av4*x[4]+av5*x[5]-(av4+av5)*x[6]

def stat_sig(temp_df,temp_vars):
    df_dummies=pd.get_dummies(temp_df)
    df_dummies.drop('genotype_pWHI5-WHI5',axis=1,inplace=True)
#     print df_dummies.head()
    vals,cov=scipy.optimize.curve_fit(fn,np.asarray(df_dummies)[:,1:].transpose(), np.asarray(df_dummies[temp_vars[0]]),method='trf')
    print temp_vars[0], temp_vars[1], 'pWHI5, pGAL1'
    print 'Slope 1: {0}, slope 2:{1}'.format(vals[0],vals[1])
    temp_p_vals = 1-scipy.stats.norm.cdf(np.absolute(vals/np.sqrt(np.diag(cov))))[:2]
    print 'P values', temp_p_vals
    print temp_p_vals<0.025
    df_dummies=pd.get_dummies(temp_df)
    df_dummies.drop('genotype_pGAL1-WHI5',axis=1,inplace=True)
    vals,cov=scipy.optimize.curve_fit(fn,np.asarray(df_dummies)[:,1:].transpose(), np.asarray(df_dummies[temp_vars[0]]),method='trf')
    print temp_vars[0], temp_vars[1], 'pGAL1, pWHI5'
    print 'Slope 1: {0}, slope 2:{1}'.format(vals[0],vals[1])
    temp_p_vals = 1-scipy.stats.norm.cdf(np.absolute(vals/np.sqrt(np.diag(cov))))[:2]
#     print vals, 1-scipy.stats.norm.cdf(np.absolute(vals/np.sqrt(np.diag(cov))))
    print 'P values', temp_p_vals
    print temp_p_vals<0.025

In [6]:
temp_vars1=['$c1_{b,seg,norm}$', '$V_{b,ell}$']
temp_df1=df[df['Cell Type']=='Daughters'][['genotype','expt']+temp_vars1]
df_dummies=pd.get_dummies(temp_df1)

In [7]:
# temp_vars = ['$t_{G1}$','$c1_{b,seg,norm}$']
temp_vars2=[['$c1_{b,seg,norm}$', '$V_{b,ell}$'], ['$c2_{b,seg,norm}$', '$V_{b,ell}$'],
          ['$t_{G1}$', '$V_{b,ell}$'], ['$t_{budded}$', '$V_{s,ell}$'],
           ['$t_{G1}$','$c1_{b,seg,norm}$'],['$t_{G1}$','$c2_{b,seg,norm}$'],['$V_{b,ell}$','$V_{div,ell}$']]
# temp_vars1 = ['$c1_{b,seg,norm}$', '$V_{b,ell}$']
for temp_vars1 in temp_vars2:
    temp_df1=df[df['Cell Type']=='Daughters'][['genotype','expt']+temp_vars1]
    stat_sig(temp_df1,temp_vars1)

$c1_{b,seg,norm}$ $V_{b,ell}$ pWHI5, pGAL1
Slope 1: -0.00926921572608, slope 2:0.00592073489802
P values [3.35941275e-11 2.48305770e-03]
[ True  True]
$c1_{b,seg,norm}$ $V_{b,ell}$ pGAL1, pWHI5
Slope 1: -0.00334848078945, slope 2:-0.00592073492771
P values [0.01576886 0.00248679]
[ True  True]
$c2_{b,seg,norm}$ $V_{b,ell}$ pWHI5, pGAL1
Slope 1: 0.00220765076416, slope 2:-0.00162585058474
P values [1.42266827e-06 1.00815455e-02]
[ True  True]
$c2_{b,seg,norm}$ $V_{b,ell}$ pGAL1, pWHI5
Slope 1: 0.000581800182778, slope 2:0.00162585057866
P values [0.13025584 0.01008155]
[False  True]
$t_{G1}$ $V_{b,ell}$ pWHI5, pGAL1
Slope 1: -2.1319033415, slope 2:-0.337458101576
P values [0.         0.13114711]
[ True False]
$t_{G1}$ $V_{b,ell}$ pGAL1, pWHI5
Slope 1: -2.46936152379, slope 2:0.337458245645
P values [0.         0.13105816]
[ True False]
$t_{budded}$ $V_{s,ell}$ pWHI5, pGAL1
Slope 1: -0.0805879591481, slope 2:-0.233127973355
P values [0.02023265 0.00016444]
[ True  True]
$t_{budded}$ $V_{